In [1]:
import sys
import networkx as nx
from igraph import Graph as igGraph
from igraph import GraphBase
import igraph as ig
from collections import Counter
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Importa la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per convertire il grafo NetworkX in formato iGraph
def convert_to_igraph(graph):
    """
    Converte un grafo NetworkX in un grafo iGraph, assicurandosi che gli ID dei vertici siano corretti.
    
    Args:
        graph (networkx.Graph): Il grafo NetworkX da convertire.
    
    Returns:
        ig.Graph: Il grafo convertito in formato iGraph.
    """
    g = ig.Graph(directed=graph.is_directed())
    # Mappatura dei nodi a indici
    mapping = {node: idx for idx, node in enumerate(graph.nodes)}
    g.add_vertices(len(mapping))
    # Aggiungi gli archi usando gli indici mappati
    edges = [(mapping[u], mapping[v]) for u, v in graph.edges if u in mapping and v in mapping]
    g.add_edges(edges)
    # Aggiungi i nomi dei nodi come attributo
    g.vs["name"] = list(mapping.keys())
    return g

# Funzione per eseguire il clustering con Walktrap
def walktrap_clustering(graph):
    """
    Esegue il clustering sul grafo utilizzando l'algoritmo Walktrap.
    
    Args:
        graph (networkx.Graph): Il grafo NetworkX su cui eseguire il clustering.
    
    Returns:
        dict: Un dizionario con l'etichetta del cluster per ciascun nodo.
    """
    # Converti il grafo in formato iGraph
    ig_graph = convert_to_igraph(graph)
    # Esegui il clustering Walktrap
    dendrogram = ig_graph.community_walktrap()
    clusters = dendrogram.as_clustering()
    # Mappa i cluster ai nodi originali
    cluster_labels = {ig_graph.vs[idx]["name"]: cluster for idx, cluster in enumerate(clusters.membership)}
    return cluster_labels

# Funzione per sanitizzare gli attributi del grafo (nodi e archi)
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e i valori degli attributi dei nodi e degli archi
    per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)

    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Esegui il clustering con Walktrap
walktrap_labels = walktrap_clustering(graph)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in walktrap_labels.items():
    gr.nodes[node]['cluster'] = label  # Assegniamo l'etichetta del cluster al nodo

# Sanitizza gli attributi del grafo prima di esportarlo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_walktrap_clusters.gml"
nx.write_gml(gr, output_path)

# Visualizza il numero di nodi per ciascun cluster
cluster_counts = Counter(walktrap_labels.values())
